<a href="https://colab.research.google.com/github/auditee/Auditee_SahaChowdhury_Zeotap-Assesment/blob/main/Auditee_SahaChowdhury_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install scikit-learn

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

customers = pd.read_csv("/content/Customers.csv")
products = pd.read_csv("/content/Products.csv")
transactions = pd.read_csv("/content/Transactions.csv")

merged_data = pd.merge(transactions, customers, on="CustomerID").merge(products, on="ProductID")

customer_features = merged_data.groupby("CustomerID").agg({
    "TotalValue": "sum",
    "Quantity": "sum",
    "ProductID": "nunique",
    "Category": lambda x: x.mode()[0]
}).reset_index()

customer_profiles = pd.merge(customers, customer_features, on="CustomerID", how="left")

customer_profiles = pd.get_dummies(customer_profiles, columns=["Region", "Category"], drop_first=True)

customer_profiles = customer_profiles.fillna(0)

numeric_columns = customer_profiles.select_dtypes(include=np.number).columns
customer_features_scaled = customer_profiles[numeric_columns]

scaler = StandardScaler()
customer_features_scaled = scaler.fit_transform(customer_features_scaled)
similarity_matrix = cosine_similarity(customer_features_scaled)

similarity_df = pd.DataFrame(similarity_matrix, index=customer_profiles["CustomerID"], columns=customer_profiles["CustomerID"])
lookalike_results = {}
for customer_id in customer_profiles["CustomerID"][:20]:

    similar_customers = similarity_df[customer_id].sort_values(ascending=False).drop(customer_id)

    top_3 = list(similar_customers.head(3).items())
    lookalike_results[customer_id] = top_3

lookalike_df = pd.DataFrame({
    "CustomerID": lookalike_results.keys(),
    "Lookalikes": lookalike_results.values()
})

lookalike_df.to_csv("Lookalike.csv", index=False)
print("Lookalike Model complete. Results saved to Lookalike.csv.")

Lookalike Model complete. Results saved to Lookalike.csv.
